In [4]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import json

chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
chrome_driver = r"E:\부트 캠프\DailyConsume_SC\chromedriver-win64\chromedriver-win64\chromedriver.exe"

service = Service(executable_path=chrome_driver)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 연결 후 주석처리 필요
# driver.get('https://www.naver.com/')

driver.switch_to.frame('searchIframe')
time.sleep(0.5)

stores = driver.find_elements(By.CLASS_NAME, "place_bluelink")

storeInfoList = []

for store in stores:
    store.click()
    time.sleep(1.5)

    driver.switch_to.default_content()
    driver.switch_to.frame('entryIframe')

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 왓더버거머시깽이 때문에 예외 처리 합니다.
    try:
        storeName = soup.select_one('#_title > div:nth-child(1) > span:nth-child(1)').text
        storeCate = soup.select_one('#_title > div:nth-child(1) > span:nth-child(2)').text
        storePlace = soup.select_one('.place_section_content > div > div.O8qbU.tQY7D > div > a > span.LDgIH').text
        storePhone = soup.select_one('.vV_z_ > span:nth-child(1)').text
    except AttributeError:
        storeName = '정보 없음'
        storeCate = '정보 없음'
        storePlace = '정보 없음'
        storePhone = '정보 없음'

    try:
        menuText = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[3]/span').text
    except Exception:
        menuText = ''

    try:
        if "메뉴" in menuText:
            tabMenu = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[3]')
        else:
            tabMenu = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[2]')
        tabMenu.click()
        time.sleep(0.5)
    except Exception as e:
        print(f"Error clicking menu tab: {e}")
        tabMenu = None

    menuList = []
    if tabMenu:
        menuSoup = BeautifulSoup(driver.page_source, 'html.parser')
        menus = menuSoup.select('.place_section_content > ul li')

        for menu in menus:
            try:
                img_tag = menu.select_one('img')
                menuImg = img_tag.attrs['src'] if img_tag else '정보 없음'
            except AttributeError:
                menuImg = '정보 없음'

            try:
                menuName = menu.select_one('.lPzHi').text if menu.select_one('.lPzHi') else '정보 없음'
            except AttributeError:
                menuName = '정보 없음'

            try:
                menuPrice = menu.select_one('.GXS1X > em').text if menu.select_one('.GXS1X > em') else '정보 없음'
            except AttributeError:
                menuPrice = '정보 없음'

            menuDictionary = {'menuImg': menuImg, 'menuName': menuName, 'menuPrice': menuPrice}
            menuList.append(menuDictionary)

    storeDictionary = {'storeName': storeName, 'storeCategory': storeCate, 'storePlace': storePlace, 'storePhone': storePhone, 'menuList': menuList}

    print(storeDictionary)
    storeInfoList.append(storeDictionary)

    driver.switch_to.default_content()
    driver.switch_to.frame('searchIframe')

print(storeInfoList)

with open('stores.json', 'w', encoding='utf-8') as f:
    json.dump(storeInfoList, f, ensure_ascii=False, indent="\t")


{'storeName': '교장맥주', 'storeCategory': '맥주,호프', 'storePlace': '서울 마포구 어울마당로 144-1 1층', 'storePhone': '0507-1349-6594', 'menuList': [{'menuImg': 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=f320_320&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20240430_246%2F1714472310778vNyPf_PNG%2FKakaoTalk_20240429_201418289_01.png', 'menuName': '생맥주 1700원', 'menuPrice': '1,700'}, {'menuImg': 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=f320_320&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20240512_250%2F1715525462463l0LLw_PNG%2F1715525074761.png', 'menuName': '매콤달콤 염통꼬치 플레이트', 'menuPrice': '14,900'}, {'menuImg': 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=f320_320&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20231030_259%2F1698656630516VuLfw_PNG%2F%25B4%25DF%25B9%25DF%25C0%25AF%25B4%25CF%25B9%25F6%25BD%25BA.png', 'menuName': '닭발유니버스(무뼈)', 'menuPrice': '25,900'}, {'menuImg': 'https://search.pstatic.net/common/?autoRotate=true&quality=95&typ